In [9]:
# Import libraries and packages
import pandas as pd
import numpy as np
import os
import math
import tableone
from datetime import datetime, timedelta
from scipy import stats
import matplotlib.pyplot as plt
import warnings

In [10]:
path = '/home/dchanci/research/pediatric_sepsis/prediction_ml/models/results_updated'
screening_method = 'inf_phoenix'

In [11]:
col_names = ['albumin', 'base_excess', 'base_deficit', 'pco2', 'po2', 'bicarbonate', 'bilirubin_total', 'bp_dias', 'bp_sys', 'bun',
              'calcium', 'calcium_ionized', 'chloride', 'co2', 'creatinine', 'fio2', 'glucose', 'hemoglobin', 'lactic_acid', 
              'map', 'pao2_fio2', 'ph', 'platelets', 'potassium', 'ptt', 'pulse', 'pupil_left_size', 'resp', 'sodium', 'spo2', 'temp', 'wbc']

col_names_fixed = ['Albumin (g/dL)', 'Base Excess (mEq/L)', 'Base Deficit (mEq/L)', 'Arterial PaCO2 (mm Hg)', 'Arterial PaO2 (mm Hg)', 'Bicarbonate (mEq/L)', 'Bilirubin (mg/dL)', 
                   'Diastolic Blood Pressure (mm Hg)', 'Systolic Blood Pressure (mm Hg)', 'BUN (mg/dL)', 'Calcium (mg/dL)', 'Ionized Calcium (mg/dL)', 
                   'Chloride (mEq/L)', 'CO2 (mEq/L)', 'Creatinine (mg/dL)', 'FiO2 (%)', 'Glucose (mg/dL)', 'Hemoglobin (g/dL)', 'Lactic Acid (mEq/L)', 
                    'Mean Arterial Pressure (mm Hg)', 'PaO2/FiO2 Ratio (mmHg)', 'pH', 'Platelets (x10\u2079/L)', 'Potassium (mEq/L)', 'PTT (seconds)', 
                    'Heart Rate (beats per minute)', 'Pupil Left Size (mm)', 'Respiratory Rate (breaths per minute)', 'Sodium (mEq/L)', 'SpO2 (%)', 
                    'Temperature (°C)', 'WBC (x10\u2079/L)']

In [12]:
# Load data
data = pd.read_parquet(os.path.join('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_models', 'features_preimp_pivot_eg_' + screening_method + '.parquet.gzip'))
data = data[col_names]
data.columns = col_names_fixed

In [13]:
# Compute statistics
mean = [round(x,2) for x in list(np.nanmean(data, axis=0))]
mean.insert(0, "Mean")
median = [round(x,2) for x in list(np.nanmedian(data, axis=0))]
median.insert(0, "Median")
q1 = [round(x,2) for x in list(np.nanquantile(data, 0.25, axis=0))]
q1.insert(0, "Q1")
q3 = [round(x,2) for x in list(np.nanquantile(data, 0.75, axis=0))]
q3.insert(0, "Q3")
min = [round(x,2) for x in list(np.nanmin(data, axis=0))]
min.insert(0, "Min")
max = [round(x,2) for x in list(np.nanmax(data, axis=0))]
max.insert(0, "Max")
std = [round(x,2) for x in list(np.nanstd(data, axis=0))]
std.insert(0, "Std")
data.reset_index(inplace=True)
data.loc[len(data.index)] = mean
data.loc[len(data.index)] = median
data.loc[len(data.index)] = q1
data.loc[len(data.index)] = q3
data.loc[len(data.index)] = min
data.loc[len(data.index)] = max
data.loc[len(data.index)] = std
data = data.iloc[-7:]
data=data.rename(columns = {'index':'Feature'})
data = data.set_index('Feature')
data = data.T
data.to_csv(os.path.join(path, screening_method, 'features_dist_eg.csv'))
data

Feature,Mean,Median,Q1,Q3,Min,Max,Std
Albumin (g/dL),2.89,2.90,2.40,3.40,1.30,4.60,0.70
Base Excess (mEq/L),3.51,2.00,1.00,5.00,0.00,19.00,3.73
Base Deficit (mEq/L),6.56,5.00,3.00,9.00,1.00,24.00,5.18
Arterial PaCO2 (mm Hg),42.65,41.00,34.50,49.00,19.10,83.50,11.75
Arterial PaO2 (mm Hg),71.81,56.00,41.00,79.00,22.00,376.00,51.75
Bicarbonate (mEq/L),22.48,22.60,19.10,26.00,6.00,43.00,6.23
Bilirubin (mg/dL),1.22,0.40,0.30,0.90,0.10,20.20,2.40
Diastolic Blood Pressure (mm Hg),60.52,60.00,50.00,70.00,31.00,100.00,14.04
Systolic Blood Pressure (mm Hg),104.88,105.00,94.00,116.00,64.00,151.00,16.24
BUN (mg/dL),14.29,11.00,7.00,16.00,2.00,93.00,12.87


In [14]:
# Load data
data = pd.read_parquet(os.path.join('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_models', 'features_preimp_pivot_sr_' + screening_method + '.parquet.gzip'))
data = data[col_names]
data.columns = col_names_fixed

In [15]:
# Compute statistics
mean = [round(x,2) for x in list(np.nanmean(data, axis=0))]
mean.insert(0, "Mean")
median = [round(x,2) for x in list(np.nanmedian(data, axis=0))]
median.insert(0, "Median")
q1 = [round(x,2) for x in list(np.nanquantile(data, 0.25, axis=0))]
q1.insert(0, "Q1")
q3 = [round(x,2) for x in list(np.nanquantile(data, 0.75, axis=0))]
q3.insert(0, "Q3")
min = [round(x,2) for x in list(np.nanmin(data, axis=0))]
min.insert(0, "Min")
max = [round(x,2) for x in list(np.nanmax(data, axis=0))]
max.insert(0, "Max")
std = [round(x,2) for x in list(np.nanstd(data, axis=0))]
std.insert(0, "Std")
data.reset_index(inplace=True)
data.loc[len(data.index)] = mean
data.loc[len(data.index)] = median
data.loc[len(data.index)] = q1
data.loc[len(data.index)] = q3
data.loc[len(data.index)] = min
data.loc[len(data.index)] = max
data.loc[len(data.index)] = std
data = data.iloc[-7:]
data=data.rename(columns = {'index':'Feature'})
data = data.set_index('Feature')
data = data.T
data.to_csv(os.path.join(path, screening_method, 'features_dist_sr.csv'))
data

Feature,Mean,Median,Q1,Q3,Min,Max,Std
Albumin (g/dL),3.02,3.00,2.50,3.50,1.40,4.60,0.69
Base Excess (mEq/L),2.91,2.00,1.00,4.00,0.00,16.00,3.06
Base Deficit (mEq/L),6.14,5.00,3.00,8.00,1.00,24.00,4.92
Arterial PaCO2 (mm Hg),41.55,39.80,33.90,47.50,19.10,83.60,11.31
Arterial PaO2 (mm Hg),72.72,62.00,47.50,82.00,24.00,299.00,42.07
Bicarbonate (mEq/L),22.30,22.60,19.30,25.60,6.00,39.60,5.65
Bilirubin (mg/dL),0.86,0.40,0.20,0.60,0.10,14.10,1.77
Diastolic Blood Pressure (mm Hg),60.43,60.00,50.00,70.00,30.00,97.00,13.95
Systolic Blood Pressure (mm Hg),104.11,104.00,94.00,114.00,66.00,144.00,14.70
BUN (mg/dL),10.93,9.00,6.00,13.00,1.00,65.00,8.34


In [16]:
# Organize tables 
dist_eg = pd.read_csv(os.path.join(path, screening_method, 'features_dist_eg.csv'))
dist_eg.rename(columns={'Unnamed: 0': 'Feature'}, inplace=True)
for i in range(dist_eg.shape[0]):
    dist_eg.loc[dist_eg.index == i, 'Mean (SD)'] = str(dist_eg.loc[dist_eg.index == i, 'Mean'].values[0]) + ' (' + str(dist_eg.loc[dist_eg.index == i, 'Std'].values[0]) + ')'
    dist_eg.loc[dist_eg.index == i, 'Median (Q1, Q3)'] = str(dist_eg.loc[dist_eg.index == i, 'Median'].values[0]) + ' (' + str(dist_eg.loc[dist_eg.index == i, 'Q1'].values[0]) + ', ' + str(dist_eg.loc[dist_eg.index == i, 'Q3'].values[0]) + ')'
    dist_eg.loc[dist_eg.index == i, 'Min, Max'] = str(dist_eg.loc[dist_eg.index == i, 'Min'].values[0]) + ', ' + str(dist_eg.loc[dist_eg.index == i, 'Max'].values[0])
dist_eg.drop(['Mean', 'Median', 'Q1', 'Q3', 'Min', 'Max', 'Std'], axis=1, inplace=True)


dist_sr = pd.read_csv(os.path.join(path, screening_method, 'features_dist_sr.csv'))
dist_sr.rename(columns={'Unnamed: 0': 'Feature'}, inplace=True)
for i in range(dist_sr.shape[0]):
    dist_sr.loc[dist_sr.index == i, 'Mean (SD)'] = str(dist_sr.loc[dist_sr.index == i, 'Mean'].values[0]) + ' (' + str(dist_sr.loc[dist_sr.index == i, 'Std'].values[0]) + ')'
    dist_sr.loc[dist_sr.index == i, 'Median (Q1, Q3)'] = str(dist_sr.loc[dist_sr.index == i, 'Median'].values[0]) + ' (' + str(dist_sr.loc[dist_sr.index == i, 'Q1'].values[0]) + ', ' + str(dist_sr.loc[dist_sr.index == i, 'Q3'].values[0]) + ')'
    dist_sr.loc[dist_sr.index == i, 'Min, Max'] = str(dist_sr.loc[dist_sr.index == i, 'Min'].values[0]) + ', ' + str(dist_sr.loc[dist_sr.index == i, 'Max'].values[0])
dist_sr.drop(['Mean', 'Median', 'Q1', 'Q3', 'Min', 'Max', 'Std'], axis=1, inplace=True)
dist_sr

dist_eg = dist_eg.merge(dist_sr, on='Feature')
dist_eg.columns = [['Feature', 'Derivation', 'Derivation', 'Derivation', 'Validation', 'Validation', 'Validation'], [' ', 'Mean (SD)', 'Median (Q1, Q3)', 'Min, Max', 'Mean (SD)', 'Median (Q1, Q3)', 'Min, Max']]
dist_eg.to_csv(os.path.join(path, screening_method, 'features_dist.csv'), index=False)
dist_eg

Feature       Derivation  \
                                                 Mean (SD)   
0                          Albumin (g/dL)       2.89 (0.7)   
1                     Base Excess (mEq/L)      3.51 (3.73)   
2                    Base Deficit (mEq/L)      6.56 (5.18)   
3                  Arterial PaCO2 (mm Hg)    42.65 (11.75)   
4                   Arterial PaO2 (mm Hg)    71.81 (51.75)   
5                     Bicarbonate (mEq/L)     22.48 (6.23)   
6                       Bilirubin (mg/dL)       1.22 (2.4)   
7        Diastolic Blood Pressure (mm Hg)    60.52 (14.04)   
8         Systolic Blood Pressure (mm Hg)   104.88 (16.24)   
9                             BUN (mg/dL)    14.29 (12.87)   
10                        Calcium (mg/dL)      8.46 (0.85)   
11                Ionized Calcium (mg/dL)      4.75 (0.66)   
12                       Chloride (mEq/L)    109.33 (7.24)   
13                            CO2 (mEq/L)     21.78 (5.46)   
14                     Creatinine (mg/dL)       0.8 (1.17)   
15                               FiO2 (%)    47.61 (28.56)   
16                        Glucose (mg/dL)   141.55 (69.85)   
17                      Hemoglobin (g/dL)     10.84 (2.18)   
18                    Lactic Acid (mEq/L)      5.24 (7.72)   
19         Mean Arterial Pressure (mm Hg)    72.46 (13.58)   
20                 PaO2/FiO2 Ratio (mmHg)  156.65 (107.46)   
21                                     pH       7.32 (0.1)   
22                     Platelets (x10⁹/L)  242.96 (136.66)   
23                      Potassium (mEq/L)       4.0 (0.77)   
24                          PTT (seconds)     39.36 (17.8)   
25          Heart Rate (beats per minute)   118.04 (27.21)   
26                   Pupil Left Size (mm)      2.97 (0.89)   
27  Respiratory Rate (breaths per minute)    27.26 (10.86)   
28                         Sodium (mEq/L)    140.98 (6.15)   
29                               SpO2 (%)     97.55 (3.48)   
30                       Temperature (°C)      36.81 (0.6)   
31                           WBC (x10⁹/L)     11.56 (7.29)   

                                              Validation  \
          Median (Q1, Q3)      Min, Max        Mean (SD)   
0          2.9 (2.4, 3.4)      1.3, 4.6      3.02 (0.69)   
1          2.0 (1.0, 5.0)     0.0, 19.0      2.91 (3.06)   
2          5.0 (3.0, 9.0)     1.0, 24.0      6.14 (4.92)   
3       41.0 (34.5, 49.0)    19.1, 83.5    41.55 (11.31)   
4       56.0 (41.0, 79.0)   22.0, 376.0    72.72 (42.07)   
5       22.6 (19.1, 26.0)     6.0, 43.0      22.3 (5.65)   
6          0.4 (0.3, 0.9)     0.1, 20.2      0.86 (1.77)   
7       60.0 (50.0, 70.0)   31.0, 100.0    60.43 (13.95)   
8     105.0 (94.0, 116.0)   64.0, 151.0    104.11 (14.7)   
9        11.0 (7.0, 16.0)     2.0, 93.0     10.93 (8.34)   
10         8.5 (7.9, 9.0)     5.9, 10.7      8.64 (0.74)   
11         4.9 (4.4, 5.2)      2.2, 6.1      4.71 (0.78)   
12   109.0 (105.0, 113.0)   91.0, 135.0    109.34 (6.59)   
13      22.0 (19.0, 25.0)     6.0, 37.0     22.13 (4.82)   
14         0.5 (0.3, 0.8)    0.17, 10.6      0.52 (0.37)   
15      35.0 (25.0, 60.0)   21.0, 100.0    43.79 (26.33)   
16    119.0 (96.0, 161.0)   59.0, 491.0   137.67 (65.76)   
17       10.8 (9.3, 12.4)     5.5, 16.4     11.34 (1.99)   
18         2.5 (1.3, 5.8)     0.4, 70.6       5.13 (6.5)   
19      72.0 (63.0, 82.0)   42.0, 112.0     72.65 (13.1)   
20  125.0 (77.57, 204.17)  29.47, 605.0   200.24 (99.84)   
21      7.34 (7.27, 7.39)     7.0, 7.55       7.34 (0.1)   
22   230.0 (140.0, 327.0)   16.0, 723.0  288.62 (136.71)   
23         3.9 (3.5, 4.4)      2.3, 6.6       4.02 (0.7)   
24      33.9 (29.5, 41.5)   21.9, 146.6     36.4 (11.56)   
25    118.0 (98.0, 138.0)   56.5, 183.0   119.82 (28.05)   
26         3.0 (2.0, 3.0)      1.0, 6.0      3.01 (0.86)   
27      25.0 (20.0, 33.0)     9.0, 68.0    28.08 (10.47)   
28   140.0 (137.0, 143.0)  128.0, 167.0    140.34 (5.64)   
29     99.0 (96.5, 100.0)   73.0, 100.0     97.72 (2.25)   
30   